## Mushroom Classification

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from IPython.display import display
from dateutil.parser import parse
import matplotlib.pyplot as plt

### ファイルの読み込み

In [14]:
df_data = pd.read_csv("../1_data/mushrooms.csv")
print(df_data.columns)
display(df_data.head())
display(df_data.tail())

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l
8123,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,o,c,l


In [15]:
print(df_data.shape)
print(df_data['class'].unique())

(8124, 23)
['p' 'e']


データは、データ数8124、特徴22項目からなる。分類は、毒キノコと食用キノコの２種類ある。

各特徴グループ毎に何種類のデータがあるか調べておく。

In [16]:
for col in df_data.columns:
    print(str(col) + ':')
    print(df_data[col].unique())

class:
['p' 'e']
cap-shape:
['x' 'b' 's' 'f' 'k' 'c']
cap-surface:
['s' 'y' 'f' 'g']
cap-color:
['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']
bruises:
['t' 'f']
odor:
['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
gill-attachment:
['f' 'a']
gill-spacing:
['c' 'w']
gill-size:
['n' 'b']
gill-color:
['k' 'n' 'g' 'p' 'w' 'h' 'u' 'e' 'b' 'r' 'y' 'o']
stalk-shape:
['e' 't']
stalk-root:
['e' 'c' 'b' 'r' '?']
stalk-surface-above-ring:
['s' 'f' 'k' 'y']
stalk-surface-below-ring:
['s' 'f' 'y' 'k']
stalk-color-above-ring:
['w' 'g' 'p' 'n' 'b' 'e' 'o' 'c' 'y']
stalk-color-below-ring:
['w' 'p' 'g' 'b' 'n' 'e' 'y' 'o' 'c']
veil-type:
['p']
veil-color:
['w' 'n' 'o' 'y']
ring-number:
['o' 't' 'n']
ring-type:
['p' 'e' 'l' 'f' 'n']
spore-print-color:
['k' 'n' 'u' 'h' 'w' 'r' 'o' 'y' 'b']
population:
['s' 'n' 'a' 'v' 'y' 'c']
habitat:
['u' 'g' 'm' 'd' 'p' 'w' 'l']


### 欠損値のチェック

In [17]:
# Count missing
pd.DataFrame(df_data.isnull().sum(), columns=["num of missing"])

,num of missing
class,0
cap-shape,0
cap-surface,0
cap-color,0
bruises,0
odor,0
gill-attachment,0
gill-spacing,0
gill-size,0
gill-color,0


欠損データはなさそう。
食用か毒キノコかの視点で、各特徴グループ毎の分布を調べてみる。

In [18]:
# Cross tab
for col in df_data.columns:
    if col=="class":
        continue
    print(col)
    df_c = pd.crosstab(index=df_data["class"], columns=df_data[col],margins=True)
    display(df_c)

cap-shape


cap-shape,b,c,f,k,s,x,All
class,,,,,,,
e,404,0,1596,228,32,1948,4208
p,48,4,1556,600,0,1708,3916
All,452,4,3152,828,32,3656,8124


cap-surface


cap-surface,f,g,s,y,All
class,,,,,
e,1560,0,1144,1504,4208
p,760,4,1412,1740,3916
All,2320,4,2556,3244,8124


cap-color


...

spore-print-color,b,h,k,n,o,r,u,w,y,All
class,,,,,,,,,,
e,48,48,1648,1744,48,0,48,576,48,4208
p,0,1584,224,224,0,72,0,1812,0,3916
All,48,1632,1872,1968,48,72,48,2388,48,8124


population


population,a,c,n,s,v,y,All
class,,,,,,,
e,384,288,400,880,1192,1064,4208
p,0,52,0,368,2848,648,3916
All,384,340,400,1248,4040,1712,8124


habitat


habitat,d,g,l,m,p,u,w,All
class,,,,,,,,
e,1880,1408,240,256,136,96,192,4208
p,1268,740,592,36,1008,272,0,3916
All,3148,2148,832,292,1144,368,192,8124


In [19]:
# Cross tab %
for col in df_data.columns:
    if col=="class":
        continue
    print(col)
    df_c = pd.crosstab(index=df_data["class"], columns=df_data[col],margins=True,normalize=True)
    display(df_c)

cap-shape


cap-shape,b,c,f,k,s,x,All
class,,,,,,,
e,0.049729,0.000000,0.196455,0.028065,0.003939,0.239783,0.517971
p,0.005908,0.000492,0.191531,0.073855,0.000000,0.210241,0.482029
All,0.055638,0.000492,0.387986,0.101920,0.003939,0.450025,1.000000


cap-surface


cap-surface,f,g,s,y,All
class,,,,,
e,0.192024,0.000000,0.140817,0.185130,0.517971
p,0.093550,0.000492,0.173806,0.214180,0.482029
All,0.285574,0.000492,0.314623,0.399311,1.000000


cap-color


...

spore-print-color,b,h,k,n,o,r,u,w,y,All
class,,,,,,,,,,
e,0.005908,0.005908,0.202856,0.214673,0.005908,0.000000,0.005908,0.070901,0.005908,0.517971
p,0.000000,0.194978,0.027573,0.027573,0.000000,0.008863,0.000000,0.223043,0.000000,0.482029
All,0.005908,0.200886,0.230428,0.242245,0.005908,0.008863,0.005908,0.293944,0.005908,1.000000


population


population,a,c,n,s,v,y,All
class,,,,,,,
e,0.047267,0.035451,0.049237,0.108321,0.146726,0.130970,0.517971
p,0.000000,0.006401,0.000000,0.045298,0.350566,0.079764,0.482029
All,0.047267,0.041851,0.049237,0.153619,0.497292,0.210734,1.000000


habitat


habitat,d,g,l,m,p,u,w,All
class,,,,,,,,
e,0.231413,0.173314,0.029542,0.031512,0.016741,0.011817,0.023634,0.517971
p,0.156081,0.091088,0.072871,0.004431,0.124077,0.033481,0.000000,0.482029
All,0.387494,0.264402,0.102413,0.035943,0.140817,0.045298,0.023634,1.000000


In [20]:
# Check how many unknown instances denoted by "?" are in stalk-root column
missdata = df_data['stalk-root'].isin(['?']).sum()
print(missdata)

2480


キノコの根の形状(Stalk-root)が不明で '?' と記されているインスタンスが2480個ある。

### データ変換
値が文字なので、統計解析しやすいように各特徴毎のone-hot形式のデータに変換する

In [21]:
df_str = df_data.copy()
for col in df_data.columns:
    col_str = col+"-str"
    df_str[col_str] = df_data[col].astype(str).map(lambda x: col+'-'+x)
    if col == "class":
        df_new = pd.get_dummies(df_str[col_str])
    else:
        df_new = pd.concat([df_new, pd.get_dummies(df_str[col_str])], axis = 1)
print(df_new.columns)
display(df_new.head())
display(df_new.tail())

Index(['class-e', 'class-p', 'cap-shape-b', 'cap-shape-c', 'cap-shape-f',
       'cap-shape-k', 'cap-shape-s', 'cap-shape-x', 'cap-surface-f',
       'cap-surface-g',
       ...
       'population-s', 'population-v', 'population-y', 'habitat-d',
       'habitat-g', 'habitat-l', 'habitat-m', 'habitat-p', 'habitat-u',
       'habitat-w'],
      dtype='object', length=119)


,class-e,class-p,cap-shape-b,cap-shape-c,cap-shape-f,cap-shape-k,cap-shape-s,cap-shape-x,cap-surface-f,cap-surface-g,...,population-s,population-v,population-y,habitat-d,habitat-g,habitat-l,habitat-m,habitat-p,habitat-u,habitat-w
0,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


,class-e,class-p,cap-shape-b,cap-shape-c,cap-shape-f,cap-shape-k,cap-shape-s,cap-shape-x,cap-surface-f,cap-surface-g,...,population-s,population-v,population-y,habitat-d,habitat-g,habitat-l,habitat-m,habitat-p,habitat-u,habitat-w
8119,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8120,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
8121,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8122,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
8123,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


各特徴グループにおいて選択肢が２つしかないタイプのグループは、One-hot化されたので片方を削除しておく。その際、上記で検出したstalk-root = '?'の列と選択肢が１つしかないveil-typeも削除して解析対象から外しておく。

In [22]:
df_new_clean = df_new.drop(["stalk-root-?", "class-e", "bruises-t", "gill-attachment-f","gill-spacing-w", "gill-size-n", 
                               "stalk-shape-t", "veil-type-p"], axis = 1)
display(df_new_clean.head())
display(df_new_clean.tail())

,class-p,cap-shape-b,cap-shape-c,cap-shape-f,cap-shape-k,cap-shape-s,cap-shape-x,cap-surface-f,cap-surface-g,cap-surface-s,...,population-s,population-v,population-y,habitat-d,habitat-g,habitat-l,habitat-m,habitat-p,habitat-u,habitat-w
0,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0


,class-p,cap-shape-b,cap-shape-c,cap-shape-f,cap-shape-k,cap-shape-s,cap-shape-x,cap-surface-f,cap-surface-g,cap-surface-s,...,population-s,population-v,population-y,habitat-d,habitat-g,habitat-l,habitat-m,habitat-p,habitat-u,habitat-w
8119,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
8120,0,0,0,0,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,0
8121,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
8122,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
8123,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0


相関係数を求めてみる

In [23]:
df_new_clean.corr().style.background_gradient().format('{:.2f}')

### データ解析

abs(相関係数) >= 0.45のグループを説明変数に設定してロジスティック回帰を実行してみる

学習データ(60%)と検証データ(40%)を分割して用意する。最後のモデルの精度をレポートする。

In [24]:
# 目的変数と説明変数を設定
y = ((df_new_clean["class-p"] > 0) * 1).values
x = df_new_clean[["bruises-f", "odor-f", "odor-n", "gill-size-b", "gill-color-b", "stalk-surface-above-ring-k", 
                  "stalk-surface-above-ring-s", "stalk-surface-below-ring-k", "ring-type-l", "ring-type-p", 
                  "spore-print-color-h"]]

# 学習データと検証データを分離
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

print(len(x_train))
print(len(x_test))

model = LogisticRegression()
print(model)
model.fit(x_train, y_train)

pred = model.predict(x_test)

# モデル精度
score = accuracy_score(y_test, pred)
print('score: %s' % score)
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

4874
3250
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
score: 0.9769230769230769
             precision    recall  f1-score   support

          0       0.98      0.98      0.98      1669
          1       0.98      0.97      0.98      1581

avg / total       0.98      0.98      0.98      3250

[[1635   34]
 [  41 1540]]
